In [4]:
import json
path = "../wiki-18.jsonl/data00/jiajie_jin/flashrag_indexes/wiki_dpr_100w/wiki_dump.jsonl"

import json

def take_first_n(path, n):
    decoder = json.JSONDecoder()
    items, buf, count = [], '', 0

    with open(path, 'r', encoding='utf-8') as f:
        # read until we have n items or hit EOF
        while count < n:
            chunk = f.read(4096)
            if not chunk:
                break
            buf += chunk

            while count < n:
                buf = buf.lstrip()
                if buf.startswith('['):        # skip opening bracket once
                    buf = buf[1:]
                    buf = buf.lstrip()
                if not buf:
                    break
                if buf[0] == ']':              # reached end of array
                    return items

                try:
                    obj, idx = decoder.raw_decode(buf)
                except json.JSONDecodeError:
                    # need more data
                    break

                items.append(obj)
                count += 1
                buf = buf[idx:].lstrip()
                if buf.startswith(','):
                    buf = buf[1:]              # consume comma before next item

        return items

first = take_first_n(path, 100000)

# write to file
with open("data/original/wiki1.jsonl", "w", encoding="utf-8") as f:
    for item in first:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Max characters in a chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,  # How to measure chunk length (default for character-based)
    separators=[" ", ""]  # Priority of split characters
)

splits = []
for i, doc in enumerate(first):
    split = text_splitter.split_text(doc["contents"])
    splits.extend(split)

chunked_data = []
for i, chunk in enumerate(splits):
    chunked_data.append({"id": i, "contents": chunk})

with open("data/chunked/wiki1_chunked.jsonl", "w", encoding="utf-8") as f:
    for item in chunked_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")


In [ ]:

objs = []
with open('data/chunked/wiki1_chunked.jsonl', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)   # parse one JSON object per line
            objs.append(obj)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {line_num}: {e}")


print(objs[:2])

[{'id': 0, 'contents': '"Evan Morris"\nEvan Morris Evan L. Morris (January 26, 1977 – July 9, 2015) was a lobbyist for Genentech and its parent corporation Roche in Washington, D.C. He began his career in Washington as an'}, {'id': 1, 'contents': 'D.C. He began his career in Washington as an intern in the Clinton White House at age 18. He began his lobbying work at Patton Boggs before moving on to Roche in 2005. His early work at Roche'}]
